In [1]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow as tf

img_folder = 'img_align_celeba'
imagens_atributes = 'celeb_attributes.csv'
videos_table_path_validation = 'celeb_bbox.csv'
max_cases_per_split = 10000
preprocess_training_test = False
preprocess_validation = True
input_size = (128,128)

In [2]:
input_size = (128,128)
images = pd.read_csv(imagens_atributes, sep=";")
bbox = pd.read_csv(videos_table_path_validation, sep=";")
data = images.merge(bbox)

# BALANCE
male = data[data['Male']==1]
female = data[data['Male']!=1].head(len(male))
data = pd.concat([male, female], ignore_index=True)

#SHUFFLE
data = data.sample(frac=1)

def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)    
    return img

def process_path(index):
    file_path = img_folder + '\\' + data.iloc[index]['file']
    img = tf.io.read_file(file_path, )
    img = decode_img(img)
    #(x,y,w,h) = data.iloc[index][['x_1','y_1','width', 'height']].values
    #img = img[x:x+w,y:y+h]
    #img = cv.resize(img, input_size)
    return img

def process_dataframe(dataframe):
    xs = []
    ys = []
    with tf.device('/cpu:0'):
        for i,row in dataframe.iterrows():
            label = row['Male'] == 1
            xs.append(process_path(i))
            ys.append(label)
        return tf.data.Dataset.from_tensor_slices((xs,ys))

In [3]:
IMG_SHAPE = (218, 178, 3)
base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = True

model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

# x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
# x = tf.keras.layers.Dense(1024, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.5)(x)
# x = tf.keras.layers.Dense(512, activation='relu')(x)
# x = tf.keras.layers.Dense(1, activation='sigmoid')(x)


# model = tf.keras.Model(base_model.input, x)
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'), tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')]) #

In [5]:
model.save('cnn_inceptionv3_pretrainned_gender_recognition_v3')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: cnn_inceptionv3_pretrainned_gender_recognition_v3\assets


In [6]:
model = tf.keras.models.load_model('cnn_inceptionv3_pretrainned_gender_recognition_v3')

In [8]:
len(list(test_ds))

3167

In [4]:
count = 0
split_count = int(len(data)/max_cases_per_split)
for split in np.array_split(data, split_count):
    print('Split' + str(count))
    data_ds = process_dataframe(split)
    train_count = int(len(split) * 0.7)
    train_ds = data_ds.take(train_count)
    test_ds = data_ds.skip(train_count)
    train_batches = train_ds.shuffle(100).batch(50)
    history = model.fit(train_batches, epochs=200)#, use_multiprocessing=False, callbacks=[ tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)])
    print('####################################################################################')
    print('#############################Split' + str(count) + 'Test ###########################')
    print('####################################################################################')
    test_loss, test_acc, test_prec, test_rec = model.evaluate(test_ds, verbose=0)
    print('Test accuracy:' + str( test_acc) + 'Test precision:' + str( test_prec) + 'Test recall:' + str( test_rec))
    print('####################################################################################')
    
    model.save('cnn_inceptionv3_pretrainned_gender_recognition_v3')
    count += 1

Split0
Epoch 1/200
148/148 [==============================] - 40s 268ms/step - loss: 0.6943 - accuracy: 0.5050 - precision: 0.4915 - recall: 0.0079
Epoch 2/200
148/148 [==============================] - 36s 241ms/step - loss: 0.6926 - accuracy: 0.5053 - precision: 0.6667 - recall: 5.4705e-04
Epoch 3/200
148/148 [==============================] - 36s 242ms/step - loss: 0.6912 - accuracy: 0.5103 - precision: 0.6105 - recall: 0.0287
Epoch 4/200
148/148 [==============================] - 36s 241ms/step - loss: 0.6889 - accuracy: 0.5207 - precision: 0.6837 - recall: 0.0585
Epoch 5/200
148/148 [==============================] - 36s 242ms/step - loss: 0.6866 - accuracy: 0.5302 - precision: 0.6782 - recall: 0.0963
Epoch 6/200
148/148 [==============================] - 36s 242ms/step - loss: 0.6834 - accuracy: 0.5474 - precision: 0.6921 - recall: 0.1537
Epoch 7/200
148/148 [==============================] - 36s 244ms/step - loss: 0.6735 - accuracy: 0.5716 - precision: 0.7749 - recall: 0.1893
Ep

ValueError: in user code:

    c:\users\red\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\keras\engine\training.py:941 test_function  *
        outputs = self.distribute_strategy.run(
    c:\users\red\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\red\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\red\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\red\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\keras\engine\training.py:909 test_step  **
        y_pred = self(x, training=False)
   

In [3]:
split_len = int(len(data)/max_cases_per_split)
split = np.array_split(data, split_len)[2]
data_ds = process_dataframe(split)
train_count = int(split_len * 0.7)
train_ds = data_ds.take(train_count)
test_ds = data_ds.skip(train_count)

In [4]:
features = np.array([list(x[0].numpy()) for x in list(data_ds)])
labels = np.array([x[1].numpy() for x in list(data_ds)])

In [9]:
img = np.expand_dims( features[1], 0)
model.predict(img)

array([[1.]], dtype=float32)

In [15]:
r = [int(x[1]) for x in list(test_ds)]

In [16]:
([x[1].numpy() for x in list(test_ds)])

array([0, 0, 0, ..., 1, 0, 1])

In [18]:
features = np.array([list(x[0].numpy()) for x in list(test_ds)])
labels = np.array(r)
test_loss, test_acc, test_prec, test_rec = model.evaluate(x=features, y=labels, verbose=0)

print('\nTest accuracy:', test_acc)
print('\nTest precision:', test_prec)
print('\nTest recall:', test_rec)


Test accuracy: 0.4947900176048279

Test precision: 0.4970684051513672

Test recall: 0.479572594165802


In [61]:
def load_caffe_model():
    return cv.dnn.readNetFromCaffe('deploy_gender.prototxt', 'gender_net.caffemodel')

gender_net = load_caffe_model()

In [65]:
preds = []
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
for img in features[:100]:
    blob = cv.dnn.blobFromImage(img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()
    preds.append(gender_preds[0].argmax())

array([[0.01312369, 0.98687637]], dtype=float32)

In [68]:
labels[:10]

array([False,  True, False, False, False, False,  True,  True, False,
        True])